## Note: 
 In this example code, Python is used for image processing (registration, skull stripping and N4 bias correction via Nipype), applying pretrained PyTorch models (CNN for whole tumor segmentation, CNN classifier for IDH status prediction), and extraction of loci and shape features. 
  R is used for application of the pretrained radiomics classifier to shpae and loci features, and application of the pretrained logistic model that connects the predicted probability from CNN classifier and radiomics classfier and yield the final probability for IDH status, which is the final step of hybrid model. 
  
  
### Python module requirements 
#### 1. Nipype 
#### 2. FSL
#### 3. ANTs
#### 4. PyRadiomics
#### 5. PyTorch
#### 6. rpy2 (with R)

In [2]:
from img_processing import *

# Image processing

#### Resample T1C to 1mm isovoxel

In [3]:
T1C_original_file = 'example_t1c.nii.gz'            # file path of original T1C 
T2_original_file = 'example_t2.nii.gz'
FLAIR_original_file = 'example_flair.nii.gz'

#### Isovoxel resampling, registration, skull stripping, bias correction, normalization, and resizing

In [4]:
# filename for isovoxel registered images
T1C_iso_file = 't1c_isovoxel.nii.gz'
T2_iso_file = 't2_isovoxel.nii.gz'
FLAIR_iso_file = 'flair_isovoxel.nii.gz'
# filename for skull-stripped images   
T1C_bet_file = 't1c_bet.nii.gz'
T2_bet_file = 't2_bet.nii.gz'
FLAIR_bet_file = 'flair_bet.nii.gz'
# filename for bias-corrected images   
T1C_corrected_file = 't1c_corrected.nii.gz'
T2_corrected_file = 't2_corrected.nii.gz'
FLAIR_corrected_file = 'flair_corrected.nii.gz'
## filename for preliminary skull-stripped T1C                                       
T1C_bet_temp_file = 'T1C_bet0_temp.nii.gz'

(t1c_unet_arr, flair_unet_arr, cropdown_info) = func_img_proc(T1C_original_file, T2_original_file, FLAIR_original_file, 
                                                              T1C_iso_file, T2_iso_file, FLAIR_iso_file,
                                                              T1C_bet_file, T2_bet_file, FLAIR_bet_file,
                                                              T1C_corrected_file, T2_corrected_file, FLAIR_corrected_file,
                                                              T1C_bet_temp_file)

brain_mask_file = T1C_bet_temp_file[:len(T1C_bet_temp_file)-len('.nii.gz')] + '_mask.nii.gz'
    

resampling T1C - completed
register T2 to T1C_isovoxel - completed
register FLAIR to T1C_isovoxel - completed
191129-10:52:31,681 interface INFO:
	 stdout 2019-11-29T10:52:31.681127:Could not find a supported file with prefix "T1C_bet0_temp_tmp_unbiased_forskull_skull"
Acquired BET mask...
Skull stripping of T1C, T2, FLAIR... - done
T1C bias correction done...
T2 bias correction done...
FLAIR bias correction done...
Image SI normalization & resizing :  done...


####  Model 1 : Automatic tumor segmentation  

In [5]:
predmask_arr = func_get_predmask(t1c_unet_arr, flair_unet_arr)

Calling pretrained Model 1...
Acquiring predicted tumor segmentation mask...


In [6]:
cropdown_info

,res_orig,x_min2,x_max2,x_len,y_min2,y_max2,y_len,z_min2,z_max2,z_len
0,"(172, 172, 140)",31,203,240,29,201,240,16,156,170


In [7]:
predmask_isovoxel_arr = func_mask_back2iso(predmask_arr, cropdown_info)
predmask_isovoxel_arr_sitk = np.transpose(predmask_isovoxel_arr, (2,1,0))
predmask_isovoxel_img = sitk.GetImageFromArray(predmask_isovoxel_arr_sitk)

predmask_isovoxel_file = 'predmask_isovoxel.nii.gz' #filename for predicted mask of isovoxel resolution
sitk.WriteImage(predmask_isovoxel_img, predmask_isovoxel_file)   # save the automatic segmentation of isovoxel resolution

In [8]:
 ### normalization for ResNet  -> resize to 128 x 128 x 128 
t1c_corrected_img = nb.load(T1C_corrected_file)
t1c_corrected_arr = t1c_corrected_img.get_data()
t2_corrected_img = nb.load(T2_corrected_file)
t2_corrected_arr = t2_corrected_img.get_data()

brain_mask = nb.load(brain_mask_file)
brain_mask_arr = brain_mask.get_data()

t1c_resnet_arr = func_norm_resnet(t1c_corrected_arr, predmask_isovoxel_arr, brain_mask_arr, cropdown_info)
t2_resnet_arr = func_norm_resnet(t2_corrected_arr, predmask_isovoxel_arr, brain_mask_arr, cropdown_info)

    

#### Model 2 : CNN classifier for IDH status prediction

In [11]:

from resnet_model import *    
def get_IDH_pred(t1c_resnet_arr, t2_resnet_arr, mask_arr):
    
    print("Calling pretrained Model 2...")
    model_resnet = ResNet(3, BasicBlock, [3,4,6,3])  
    model_filename = 'MODEL2_ResNet_CNNclassifier.pth'
    checkpoint = torch.load(model_filename)
    model_resnet.load_state_dict(checkpoint['model_state_dict'])
    model_resnet.eval()
    model_resnet.cuda()
    
    print("Calculating a predicted probabilitiy...")
    x_arr = get_maxROI(t1c_resnet_arr, t2_resnet_arr, mask_arr)
    
    x_arr = torch.from_numpy(x_arr).float()
    x_arr = x_arr.cuda()
    
    with torch.no_grad():
        output = model_resnet(x_arr)
    
    output = nn.Softmax(dim=1)(output)
    
    output_mean = torch.sum(output[:,1])/5
    
    return output, output_mean
    

In [12]:
resnet_prob, resnet_prob_mean = get_IDH_pred(t1c_resnet_arr, t2_resnet_arr, predmask_arr)

Calling pretrained Model 2...
Calculating a predicted probabilitiy...


### Model 3 : radiomics classifier 

###### Shape and location feature extraction


In [15]:
df_shape = func_shape(predmask_arr)
df_shape

,Shape_3d_Elongation,Shape_3d_Flatness,Shape_3d_LeastAxisLength,Shape_3d_MajorAxisLength,Shape_3d_Maximum2DDiameterColumn,Shape_3d_Maximum2DDiameterRow,Shape_3d_Maximum2DDiameterSlice,Shape_3d_Maximum3DDiameter,Shape_3d_MeshVolume,Shape_3d_MinorAxisLength,Shape_3d_Sphericity,Shape_3d_SurfaceArea,Shape_3d_SurfaceVolumeRatio,Shape_3d_VoxelVolume
0,0.923991,0.635154,21.481904,33.821559,38.832976,35.510562,37.64306,41.737274,15695.625,31.250816,0.830737,3649.274091,0.232503,15730.0


In [16]:
df_loci = func_loci(T1C_iso_file, predmask_isovoxel_file)   # filepaths to T1C_isovoxel and predicted mask from Model 1 that is reconstructed back to isovoxel space


Acquiring tumor loci information...
True
/home/baon/PycharmProjects/0_DeepLearning_glioma/hybridIDH_github_publication/mask_2mni.nii.gz
atlasquery -a "MNI Structural Atlas" -m /home/baon/PycharmProjects/0_DeepLearning_glioma/hybridIDH_github_publication/mask_2mni.nii.gz
Cerebellum:8.0845
Occipital Lobe:0.2472
Temporal Lobe:51.2400

3
Cerebellum:8.0845
10
Occipital Lobe:0.2472
14
Temporal Lobe:51.2400
13
{'mni_Temporal Lobe': 51.24, 'mni_Occipital Lobe': 0.2472, 'mni_Cerebellum': 8.0845}


In [17]:
sla_features = pd.concat([df_shape, df_loci], axis =1)

# Add patient's age
sla_features['age'] = pd.Series(50)  ## change 50 to the patient's age.

# Concatenate the predicted probability from CNN classifier (Model 2)
sla_features['resnet_prob'] = pd.Series(resnet_prob_mean.cpu().numpy())


sla_features.to_csv('./sla_features.csv')  ## file path to shape_loci_age feature table

In [18]:
sla_features

,Shape_3d_Elongation,Shape_3d_Flatness,Shape_3d_LeastAxisLength,Shape_3d_MajorAxisLength,Shape_3d_Maximum2DDiameterColumn,Shape_3d_Maximum2DDiameterRow,Shape_3d_Maximum2DDiameterSlice,Shape_3d_Maximum3DDiameter,Shape_3d_MeshVolume,Shape_3d_MinorAxisLength,...,mni_Cerebellum,mni_Frontal Lobe,mni_Insula,mni_Occipital Lobe,mni_Parietal Lobe,mni_Putamen,mni_Temporal Lobe,mni_Thalamus,age,resnet_prob
0,0.923991,0.635154,21.481904,33.821559,38.832976,35.510562,37.64306,41.737274,15695.625,31.250816,...,8.0845,0,0,0.2472,0,0,51.24,0,50,0.9999998


### R for radiomics classfier and logit model -> final predicted probability of hybrid model

In [ ]:
%load_ext rpy2.ipython


In [ ]:
%%R -i df -w 5 -h 5 --units in -r 200

In [ ]:
sla_features <- read.csv("./sla_features.csv", header=T, sep=",") # file path to the table with shape+loci+age+ prob from Model2 (CNN classifier)
radiomics_classifier <- readRDS("./radiomics_classifier.rds")
mylogit <- readRDS("./hybrid_logit.rds")


sla_prob <- predict(radiomics_classifier, newdata = sla_features, type = "prob")[2]
sla_prob <- as.numeric(sla_prob)
resnet_prob <- sla_features$resnet_prob
table_test <- as.data.frame(cbind(sla_prob, resnet_prob))
final_prob <- predict(mylogit, newdata=table_test, type="response")[1]
print(paste("final probability for IDH mutation is :", round(final_prob,3)))
